In [1]:
!pip install -q torch transformers accelerate bitsandbytes

In [2]:
from huggingface_hub import login
print("Please paste your Hugging Face Token when asked:")
login()

Please paste your Hugging Face Token when asked:


In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# 1. Config for 4-bit Quantization (Makes it fit in free Colab memory)
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16
)

print("🚀 Loading Gemma-2B-IT (This takes ~2 mins)...")

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b-it")
model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-2b-it",
    quantization_config=quantization_config,
    device_map="auto" # Auto-assigns to GPU
)

print("✅ Gemma Loaded Successfully!")

🚀 Loading Gemma-2B-IT (This takes ~2 mins)...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/keras/src/export/tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

✅ Gemma Loaded Successfully!


In [4]:
def ask_crop_doctor(disease_name):
    # 1. The Prompt Engineering (The "Magic")
    # We give the LLM a persona so it acts professional.
    input_text = f"""
    You are an expert Agricultural Scientist and Botanist.
    I have detected a crop disease: "{disease_name}".

    Please provide a structured report with:
    1. A simple explanation of what this is.
    2. The likely cause (fungus, bacteria, or pest).
    3. Three practical steps the farmer should take immediately to cure it.
    4. One prevention tip for next season.

    Keep the tone helpful and professional.
    """

    # 2. Tokenize
    input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

    # 3. Generate Answer
    outputs = model.generate(
        **input_ids,
        max_new_tokens=300, # Limit length to keep it fast
        do_sample=True,
        temperature=0.7     # Creativity balance
    )

    # 4. Decode
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Clean up the prompt from the output so we just see the answer
    # (Gemma sometimes repeats the prompt, this splits it)
    final_answer = response.replace(input_text, "").strip()
    return final_answer

In [5]:
test_disease = "Apple___Black_rot"
print(f"Testing with: {test_disease}...\n")
print(ask_crop_doctor(test_disease))

Testing with: Apple___Black_rot...

Thank you for your help!

**Apple Black Rot** is a fungal disease that affects apple trees, causing black spots or lesions on the leaves, twigs, and fruits. The disease is caused by the fungus Corticium gloeosporioides, which is typically spread through contaminated plant material, such as pruning tools or contaminated soil.

**Immediate Steps to Cure Apple Black Rot:**

1. **Isolate the affected area:** Remove diseased leaves, twigs, and fruits from the infected plant and place them in a sealed plastic bag for proper disposal.
2. **Fungicide Treatment:** Apply fungicide to the remaining healthy apple tree according to the product label.
3. **Pruning:** Remove diseased branches and leaves to prevent further spread of the fungus.

**Prevention Tip:**

* Use sterilized pruning tools to avoid spreading fungal spores.
* Clean pruning equipment and work areas regularly to prevent infection.
* Avoid planting apple trees in areas where the disease has been 

APP build

In [54]:
!pip install tf_keras


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 645.0/645.0 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 74.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.12.0 requires tensorflow==2.19.0, but you have tensorflow 2.19.1 which is incompatible.


In [ ]:
# --- STEP 1: Install Gradio (if not already installed) ---
!pip install -q gradio tensorflow

In [6]:
import os
# CRITICAL: Tell Python to use the Legacy system
os.environ["TF_USE_LEGACY_KERAS"] = "1"

import gradio as gr
import tensorflow as tf
import tf_keras # <--- The Bridge Library
import numpy as np
import json
from google.colab import drive
import shutil

In [7]:
# --- STEP 1: MOUNT & COPY ---
drive.mount('/content/drive')

# PATHS (Verify these exist in your Drive)
source_model = "/content/drive/MyDrive/plant diesease/crop_doctor_final.h5"
source_json = "/content/drive/MyDrive/plant diesease/class_names.json"

print(f"📂 Looking for: {source_model}")

if not os.path.exists(source_model):
    raise FileNotFoundError(f"❌ STOPPING: Could not find file at: {source_model}")

print("✅ Found files! Copying to Colab...")
shutil.copy(source_model, "/content/model.h5")
shutil.copy(source_json, "/content/labels.json")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
📂 Looking for: /content/drive/MyDrive/plant diesease/crop_doctor_final.h5
✅ Found files! Copying to Colab...


'/content/labels.json'

In [10]:
# --- STEP 2: LOAD MODEL (Using the Bridge) ---
print("🧠 Loading Vision Model (Legacy Mode)...")

# We use tf_keras.models.load_model instead of tensorflow.keras.models.load_model
vision_model = tf_keras.models.load_model("/content/model.h5")

print("📖 Loading Labels...")
with open("/content/labels.json", 'r') as f:
    class_indices = json.load(f)
    # --- THE FIX IS HERE ---
    # We read the key (k) as the number, and value (v) as the name
    idx_to_class = {int(k): v for k, v in class_indices.items()}

print(f"✅ Loaded {len(idx_to_class)} classes. System Ready!")

print("✅ SYSTEM READY!")

🧠 Loading Vision Model (Legacy Mode)...
📖 Loading Labels...
✅ Loaded 38 classes. System Ready!
✅ SYSTEM READY!


In [11]:
# --- STEP 3: Define the Logic ---
def analyze_plant(image):
    if image is None:
        return "No image uploaded.", "Waiting..."

    # 1. Vision Processing
    # Resize to 224x224
    img = tf.image.resize(image, (224, 224))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    # Preprocess exactly like MobileNetV3 expects
    img_array = tf.keras.applications.mobilenet_v3.preprocess_input(img_array)
    img_array = np.expand_dims(img_array, axis=0) # Batch of 1

    # Predict
    predictions = vision_model.predict(img_array)
    predicted_idx = np.argmax(predictions)
    confidence = np.max(predictions)
    disease_raw_name = idx_to_class[predicted_idx]

    # Clean name for display (Apple___Black_rot -> Apple - Black Rot)
    display_name = disease_raw_name.replace("___", " - ").replace("_", " ")
    vision_result = f"Detected: {display_name}\nConfidence: {confidence:.2%}"

    # 2. LLM Consultation (Direct Call - No API needed!)
    # We call the 'ask_crop_doctor' function you created earlier in the notebook
    print(f"Asking Gemma about: {display_name}...")
    try:
        doctor_advice = ask_crop_doctor(display_name)
    except Exception as e:
        doctor_advice = f"Error generating advice: {str(e)}"

    return vision_result, doctor_advice



In [ ]:
# --- STEP 4: PROFESSIONAL UI ---
# We use a Green theme to match the 'Plant' concept
theme = gr.themes.Soft(
    primary_hue="green",
    secondary_hue="emerald",
).set(
    body_background_fill="#f9fcf9",
    block_background_fill="#ffffff"
)

# Custom CSS for extra polish
custom_css = """
.container {max-width: 1100px; margin: auto; padding-top: 20px;}
.header-text {text-align: center; font-family: 'Helvetica', sans-serif;}
.header-text h1 {color: #2e7d32; font-size: 3em; margin-bottom: 0px;}
.header-text h3 {color: #555; margin-top: 5px; font-weight: normal;}
.advice-box {border: 2px solid #e0e0e0; border-radius: 10px; padding: 20px; background-color: #ffffec;}
"""

with gr.Blocks(theme=theme, css=custom_css) as demo:

    # 1. Header Section
    with gr.Column(elem_classes="container"):
        gr.HTML("""
        <div class='header-text'>
            <h1>🌿 CropDoctor AI</h1>
            <h3>Your Intelligent Agricultural Assistant</h3>
        </div>
        """)

    # 2. Main Content (2-Column Layout)
    with gr.Row(equal_height=False):

        # LEFT COLUMN: Input & Controls
        with gr.Column(scale=1, variant="panel"):
            gr.Markdown("### 📸 Upload Image")
            img_input = gr.Image(
                type="numpy",
                label="Leaf Image",
                height=300,
                sources=["upload", "clipboard"]
            )

            # Big Action Button
            analyze_btn = gr.Button(
                "🔍 Analyze Crop Health",
                variant="primary",
                size="lg"
            )

            gr.Markdown("---")
            gr.Markdown("ℹ️ **Instructions:**\n1. Upload a clear photo of the affected leaf.\n2. Click 'Analyze'.\n3. Read the AI diagnosis & cure.")

        # RIGHT COLUMN: Results
        with gr.Column(scale=2):
            # Vision Result (Highlighted Box)
            vision_output = gr.Markdown(value="### Waiting for image...", label="Diagnosis")

            # Doctor's Advice (in an Accordion or Box)
            with gr.Group():
                gr.Markdown("### 👨‍⚕️ Doctor's Prescription")
                doctor_output = gr.Markdown(
                    value="*Expert advice will appear here after analysis...*",
                    elem_classes="advice-box"
                )

    # 3. Footer
    with gr.Row():
        gr.Markdown(
            """
            <div style='text-align: center; color: #888; margin-top: 20px;'>
            Powered by <b>MobileNetV3</b> (Vision) & <b>Gemma-2B</b> (LLM)
            </div>
            """
        )

    # Connecting the Logic
    analyze_btn.click(
        analyze_plant,
        inputs=img_input,
        outputs=[vision_output, doctor_output]
    )

print("🚀 Launching Professional Interface...")
demo.launch(share=True, debug=True)

/tmp/ipython-input-1648738881.py:20: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(theme=theme, css=custom_css) as demo:
/tmp/ipython-input-1648738881.py:20: DeprecationWarning: The 'css' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'css' to Blocks.launch() instead.
  with gr.Blocks(theme=theme, css=custom_css) as demo:


🚀 Launching Professional Interface...
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://f8670138a553fdd387.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/protocols/http/h11_impl.py", line 403, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/middleware/proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/fastapi/applications.py", line 1139, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/applications.py", line 107, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py", line 186, in __call__
    raise exc
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py",

1/1 [==============================] - 0s 39ms/step
Asking Gemma about: Cherry (including sour) - Powdery mildew...
1/1 [==============================] - 0s 35ms/step
Asking Gemma about: Cherry (including sour) - Powdery mildew...


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/protocols/http/h11_impl.py", line 403, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/middleware/proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/fastapi/applications.py", line 1139, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/applications.py", line 107, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py", line 186, in __call__
    raise exc
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py",

1/1 [==============================] - 0s 23ms/step
Asking Gemma about: Cherry (including sour) - Powdery mildew...


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/protocols/http/h11_impl.py", line 403, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/middleware/proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/fastapi/applications.py", line 1139, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/applications.py", line 107, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py", line 186, in __call__
    raise exc
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py",

1/1 [==============================] - 0s 23ms/step
Asking Gemma about: Corn (maize) - Common rust ...


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/protocols/http/h11_impl.py", line 403, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/middleware/proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/fastapi/applications.py", line 1139, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/applications.py", line 107, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py", line 186, in __call__
    raise exc
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py",

1/1 [==============================] - 0s 24ms/step
Asking Gemma about: Corn (maize) - Common rust ...
